In [ ]:
import numpy as np
import pandas as pd

### Handling Missing Values (contd.)

In [ ]:
mark_data = pd.read_csv('./data/marketing.csv')

In [ ]:
mark_data.sample(5)


* What percentage of missing values for each column in the dataset? 




* Which attribute has the most missing values in the dataset? (**Hint**: To get the index of the maximum element you can use [`idxmax()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.idxmax.html) function)




* How do you fill the missing values with a `0`? 


* **Most Common Use**: Can you fill each missing value with the corresponding average for that attribute?
    * For example, if 'Education' attribute is missing for a person, can you find the average 'Education' of all people and fill that missing 'Age' with that average.

# Combining Pandas Datasets with Concatenation [MORE INFO](https://pandas.pydata.org/pandas-docs/stable/merging.html)

## Introduction

In [ ]:
# For this tutorial, we will need college_loan_defaults dataset.
college_loan_defaults = pd.read_csv(
    './data/college-loan-default-rates.csv', index_col='opeid')

# Keep in mind that the original dataset has this many rows
len(college_loan_defaults)

The Office of Postsecondary Education Identification (OPEID) code for each college is used as an index

In [ ]:
college_loan_defaults.head()

## `pd.concat`
You can think of the `pd.concat` function as the equivalent of the NumPy `concatenate` function for `Series` and `DataFrame` objects.

Will we spend most of our time on how these function works with `DataFrame` objects as opposed to `Series` objects since in practice that is how it is used most frequently.

When it comes to using the `pd.concat` function, the most basic question is whether you are adding *additional rows* or *additional columns*. We'll run through the function arguments based on concatenating rows and then come back for a look at how we perform column concatentations.

### Concatenating `DataFrame` Rows

In [ ]:
# Here, I'll split the college_loan_defaults into multiple 
# sections of rows that we will then stiched back together.
part_1 = college_loan_defaults.iloc[:1000]
part_2 = college_loan_defaults.iloc[1000:2000]
part_3 = college_loan_defaults.iloc[1999:]


In [ ]:
# This creates three parts:
# rows 0-999
# rows 1000-1999
# rows 1999-end -> notice 1999 appears twice
part_3.index & part_2.index

#### Basic Usage

In [ ]:
# Join all three parts together pd.concat
concatenated_dataframe = pd.concat([part_3, part_1, part_2])
concatenated_dataframe.head()

In [ ]:
print (concatenated_dataframe.shape[0])
print (part_1.shape[0], part_2.shape[0], part_3.shape[0])

<div class="alert alert-block alert-info">
Notice that `pd.concat` does not sort the elements of the DataFrame that it returns.
</div>

#### Handling Duplicate Index Values with `verify_integrity` & `ignore_index` Parameters
You probably didn't notice, but we got a school that is appearing twice in our list.

In [ ]:
# The `DataFrame.index.duplicated` function returns a boolean array
# we can use as a mask to extract duplicate records.
concatenated_dataframe[concatenated_dataframe.index.duplicated()]

In [ ]:
concatenated_dataframe.loc[1698]

**NOTE**: See how concatenated_dataframe.loc[1698] is using the explit index OPEID to select column(s). Since, we have two rows with the same index (opeid) in the concatenated_dataframe, we get two rows displayed above. 

Now, I purposefully caused this problem for us (by including the 1999 indexed element in both `part_2` and `part_3`; but in the real world this is pretty common!

Sometimes you might want to keep both entries (often the case if the index value is the same but the rest of the data is different). If so, you can pass the **`ignore_index`** parameter with a value of **`True`** to the function and **all existing index values will be destroyed** and a new one integer based one will be created for you.

In [ ]:
concatenated_dataframe = pd.concat([part_2, part_3, part_1], ignore_index=True)
concatenated_dataframe.head()

If on the other hand, a duplicate index would mean there is a data problem that you don't want to allow, you can specify the `verify_integrity` parameter as `True`.

When this is passed, the existence of duplicate indices will generate a `ValueError` exception.

In [ ]:
concatenated_dataframe = pd.concat([part_2, part_3, part_1], verify_integrity=True)

#### Handling Column Mismatches with the `join` Parameter
Sometimes you will have two sets of rows that you want to join together, but the sets don't have all of the same columns.

I'll create a couple of additional small `DataFrame` objects from our college loan dataset to demonstrate our options here.

In [ ]:
# DataFrame 1
# Contains the first 5 rows of the original dataset
# But only the name, city, and state columns
name_city_state_columns_only = college_loan_defaults[['name', 'city', 'state']].iloc[:5]

# DataFrame 2
# Contains the second 5 rows of the original dataset
# But only the name, state, and zipcode columns
name_state_zipcode_columns_only = college_loan_defaults[['name', 'state', 'zipcode']].iloc[5:10]

In [ ]:
name_city_state_columns_only

In [ ]:
name_state_zipcode_columns_only

We have have 2 sets of 5 rows that we want to concatenate together, but they have different columns. Let's see what happens if you don't specify anything with the **`join`** parameter.

In [ ]:
pd.concat([name_city_state_columns_only, name_state_zipcode_columns_only])

See how Pandas adds the special `NaN` value for any column that didn't have a value in the original dataframes? 

The other option is to drop any columns where there is not data in both sets of rows. You can do this be specifying a value of **`inner`** to the join parameter of the function.

Let's demonstrate how doing so will result in only the shared columns (name, state) appearing in the final dataframe.

In [ ]:
pd.concat([name_city_state_columns_only, name_state_zipcode_columns_only], join='inner')

### Concatenating `DataFrame` Columns
Now let's go back and see how we can use the `pd.concat` function to merge two sets of columns with the same index (row) values.

The data will start out a little dirty but we will clean it up with our parameters.

In [ ]:
# DataFrame 1
# Contains the first 5 rows of the original dataset
# But only the name, city, and state columns
name_city_state_columns = college_loan_defaults[['name', 'city', 'state']].iloc[:5]

# DataFrame 2
# Contains the 7 rows of the original dataset - this will cause a duplicate index
# But only default rates columns
default_rates = college_loan_defaults[
    ['year_1_default_rate',
     'year_2_default_rate', 
     'year_3_default_rate']].iloc[:7]

In [ ]:
name_city_state_columns

In [ ]:
default_rates

Now let's do a simple concatenation. To add columns we have to specify the `axis` parameter with a value of **`1`** or **`col`** to indicate we are adding colums, not rows.

In [ ]:
pd.concat(
    [name_city_state_columns, default_rates], 
    axis=1)

<div class="alert alert-block alert-info">
<p>
Note that the ``pd.concat()`` function is smart to match the rows based on the index `opeid`.  
</div> 

There are a couple of important things to notice here:
* Unlike when concatenating rows, this time Pandas did **sort the rows based on the index**. Just something to be aware of.
* See how there are a couple of rows with `NaN` values for their first three colums.  That's because our `name_and_default_rates` dataframe had two additional rows for which there were no corresponding values in `name_city_state_zipcode_columns`.

Let's drop the rows with `NaN` values by specifying an inner join.

In [ ]:
pd.concat(
    [name_city_state_columns, default_rates], 
    axis=1, join='inner')

Finally, let's talk about the how the **`verify_integrity`** and **`ignore_index`** parameters would work when concatenating columns.

Let's say that we had included the city column in both dataframes:
* The default behavior of `pd.concat` would have been to create a new dataframe with 2 "city" columns.
* You could make Pandas throw a `ValueError` exception by passing `verify_integrity=True` to the function.
* You could also throw out all the column names and replace them with an 0-based series of integers.  This would result in the values of "city" being duplicated in two columns, but the columns would have different integer "names".

# Combining Datasets with Merge [MORE INFO](https://pandas.pydata.org/pandas-docs/stable/merging.html)

We will be exploring another way to combine datasets through the **`pd.merge`** function.

Those who have a background in databases will find a significant amount of overlap between your SQL work and the merge function.

## The 3 Categories of Joins
There are 3 different categories of merges/joins which are defined by the characteristics of the shared columns/indices:
* One-to-One: Each shared value exists only once in both dataframes.
* One-to-Many: A given shared value exists once in first dataframe, but 1 or more times in the second dateframe.
* Many-to-Many: A given shared value exists 1 or more times in both dataframes.

Let's provide an example of each type of join from our datasets.

### One-to-One Join

<div class="alert alert-block alert-info">
<p>
This will feel pretty similar to concatenating columns.
</p>
</div>

In [ ]:
# Team Members Favorite Restaurants
team_restaurants = pd.DataFrame(
    {'restaurant': ['In-N-Out', 'Chipotle', 'Chick-Fil-A'], 
    'name': ['Mike', 'Kim', 'Roger']})
team_restaurants

In [ ]:
# Team Members Favorite Restaurants
items_locations = pd.DataFrame(
    {'items': ['Fries', 'Pizza', 'Barritos','Pasta', 'Shakes'], 
    'locations': ['Chicago', 'New York', 'San Diego', 'Pittsburgh', 'Seattle']})
items_locations

In [ ]:
# Restaurant Items
restaurant_items = pd.DataFrame(
    {
        'item': [
        'Shakes', 
        'Burritos', 
        'Burger'
        ]
    ,
        'restaurant':[
        'In-N-Out',
        'Chipotle',
        'Five Guys',
    ]
    }
)
restaurant_items

In [ ]:
team_restaurants

The **`restaurant`** field in the restuarant_items, team_restaurants dataset is a unique field, that is it the restaurant names appear only once in each dataset. 

Because of this, if we merge the two dataframes it will be a **1-1 join.**

In [ ]:
pd.merge(team_restaurants, restaurant_items)

Great. Here's what Pandas did:
1. Identified the matching column(s) between the two dataframes: **`restaurant`**.
1. Found matching **`restaurant`** values between the two dataframes.
1. Merged the columns of matching **`restuarant`** values together.
1. **Important**: Notice that a new index was generated.

<div class="alert alert-block alert-info">
<p>
In our discussion, we will reference to the columns that pandas is using to find matches between dataframes as the "join column(s)".
</p>
</div>

#### Controlling the Join Type with the `how` Parameter
Did you notice that some of the records from each of the original dataframes didn't make it into the merge product?

This is because the type of join that was applied to the dataframes was called an **inner join**.

The are actually 4 types of joins that you can use:
* **Inner Join**: To be included in the output dataframe, the join column(s) value must exist in both original dataframes. 
    * This is why some of the records didn't get included in the output, because they didn't have a corresponding join column(s) values in the other dataframe.
* **Outer Join**: All records from both dataframes are included in the output. Pandas simply fills in `NaN` where there is no corresponding join column(s) value.
* **Left Join**: All rows from the first (left) dataframe will be included in the output dataframe, regardless of whether there is a matching join column(s) value in the second (right) dataframe.
* **Right Join**: All rows from the second (right) dataframe will be included in the output dataframe, regardless of whether there is a matching join columns value in the left (first) dataframe.

Let's go ahead and try all these different types of joins to see how our output changes.

In [ ]:
team_restaurants

In [ ]:
restaurant_items

In [ ]:
# Outer Join
# All records from both dataframes are included.
# NaN is inserted into missing grid point.
pd.merge(team_restaurants, restaurant_items, how="outer")

In [ ]:
pd.merge(team_restaurants, restaurant_items, how="left")

In [ ]:
pd.merge(team_restaurants, restaurant_items, how="right")

### One-to-Many Join

In [ ]:
# Restaurant Items
restaurant_items = pd.DataFrame(
    {
        'item': [
        'Burgers', 'Fries', 'Shakes', 
        'Tacos', 'Burritos', 'Chips',
        'Chicken Sandwich', 'Fries', 'Salads'
        ]
    ,
        'rest':[
        'In-N-Out', 'In-N-Out', 'In-N-Out', 
        'Chipotle', 'Chipotle', 'Chipotle',
        'Five Guys', 'Five Guys', 'Five Guys'
    ]
    }
)
restaurant_items

In [ ]:
team_restaurants

In [ ]:
pd.merge(team_restaurants, restaurant_items)

#### Specifying the Join Columns
Well... that isn't want we wanted.

Thankfully though, the error message is pretty self-explanatory. Pandas thinks there are no common columns to merge on.

The reason for this is that the common values are held in columns with slightly different names. We have to explain to Pandas what to do when this happens by specifying the names of the columns to join on.

In [ ]:
team_restaurants

In [ ]:
restaurant_items

In [ ]:
# Use the left_on and right_on parameters to specify the
# name(s) of the join column(s) in the first(left)
# and second(right) dataframes.
pd.merge(
    team_restaurants, 
    restaurant_items,
    left_on='restaurant',
    right_on='rest')

<div class="alert alert-block alert-info">
<h5>There can be more than 1 join column</h5>
<p>
In this example, we have specified only one join column. But you can specify multiple columns if you so desire. Just pass them as a list to the `left_on` and `right_on` parameters.
</p>
</div>

### Many-to-Many Join

In [ ]:
# Team Members Favorite Restaurants
team_restaurants = pd.DataFrame(
    {'restaurant': ['In-N-Out', 'Chipotle', 'Chick-Fil-A', 'Chick-Fil-A', 'In-N-Out'], 
    'name': ['Mike', 'Kim', 'Roger', 'Sam', 'Sonia']})
team_restaurants


In [ ]:
# Restaurant Items
restaurant_items = pd.DataFrame(
    {
        'item': [
        'Burgers', 'Fries', 'Shakes', 
        'Tacos', 'Burritos', 'Chips',
        'Chicken Sandwich', 'Fries', 'Salads'
        ]
    ,
        'rest':[
        'In-N-Out', 'In-N-Out', 'In-N-Out', 
        'Chipotle', 'Chipotle', 'Chipotle',
        'Five Guys', 'Five Guys', 'Five Guys'
    ]
    }
)
restaurant_items

In [ ]:
pd.merge(team_restaurants, restaurant_items, left_on = 'restaurant', right_on = 'rest', how = "outer")

<div class="alert alert-block alert-info">
<p> You could merge two dataframes based on index as well. </p>

<p>
If you wanted to, you could actually use the index of one dataframe and a column of the other dataframe if you wanted. Pandas gives you great flexibility here. 
</p>
</div>

## Activity: Compute the population density of each state

We will learn ``pd.merge()`` operation using the three datasets from your textbook. 


* Load the datasets

In [ ]:
pop = pd.read_csv('./data/state-population.csv')
areas = pd.read_csv('./data/state-areas.csv')
abbrevs = pd.read_csv('./data/state-abbrevs.csv')

In [ ]:
pop.head()

In [ ]:
areas.head()

In [ ]:
abbrevs.head()

* Create a DataFrame named `areas_abbrvs_merged` by merging the ``areas`` DataFrame and ``abbrevs`` DataFrame to get the state names, state abbreviations, and area into one DataFrame

* Merge the DataFrame created above (`areas_abbrvs_merged`) with the `pop` DataFrame to create a the `desired_data` DataFrame

* Finally, create a column called density using the `population` and `area(sq. mi)` columns. 

* Which state has highest 'total' population density in year 2012? 

In [ ]:
# Step 1: Filter out the data with ages is 'total' and year is 2012

In [ ]:
# Step 2: Find the maximum pop density in the filtered dataframe

In [ ]:
# Step 3: Find the state that has the maximum pop density

# Writing files back 

Until now you have been loading the dataset from your computer, however, you might want to store the data back to the computer to use it later. 

For example, in the above activity were you created population density by merging bunch of DataFrames, you might want to save that DataFrame, rather than redoing all the steps. 

## pd.DataFrame.to_csv()

In [ ]:
pd.DataFrame.to_csv?

In [ ]:
desired_data.to_csv('./data/state-population-density.csv')

<div class="alert alert-block alert-info">
<p>
``to_csv()`` by default writes the index (row names) as well. This will create an additional column with the indexes. If you want to avoid it, you can use keyword parameter ``index = False`` to avoid creating a column for the index. 
</p>
</div>

In [ ]:
desired_data.to_csv('./data/state-population-density.csv', index = False)

# Hierarchical Indexing


### Multiindex

If you set an index to more than one columnn you are creating multi index or Hieararchical index. This makes asking questions based on indexes a lot more easier, and also opens the possibility of working with multidimensional data. 

We'll use the example sourced from [here](https://chrisalbon.com/python/pandas_hierarchical_data.html). 

In [ ]:
# Create dataframe
raw_data = {'regiment': ['Nighthawks', 'Nighthawks', 'Nighthawks', 'Nighthawks', 'Dragoons', 'Dragoons', 'Dragoons', 'Dragoons', 'Scouts', 'Scouts', 'Scouts', 'Scouts'], 
        'company': ['1st', '1st', '2nd', '2nd', '1st', '1st', '2nd', '2nd','1st', '1st', '2nd', '2nd'], 
        'name': ['Miller', 'Jacobson', 'Ali', 'Milner', 'Cooze', 'Jacon', 'Ryaner', 'Sone', 'Sloan', 'Piger', 'Riani', 'Ali'], 
        'preTestScore': [4, 24, 31, 2, 3, 4, 24, 31, 2, 3, 2, 3],
        'postTestScore': [25, 94, 57, 62, 70, 25, 94, 57, 62, 70, 62, 70]}
df = pd.DataFrame(raw_data, columns = ['regiment', 'company', 'name', 'preTestScore', 'postTestScore'])
df

In [ ]:
df_1_ind = df.set_index('regiment')
df_1_ind

* How do we get the average scores, based on the regiment? 

In [ ]:
df_1_ind.mean(level = 'regiment')

* How about you want to get the mean scores, based on the company but not the regiment? 

In [ ]:
# Set the hierarchical index to be by regiment, and then by company
df_2_ind = df.set_index(['regiment', 'company'])
df_2_ind

<div class="alert alert-block alert-info">
<p>
Having multiple indexes will give you an easy way to model more than two dimensional data with DataFrames, which are by default a two dimensional data structures. 
</p>
<p>
For the above example, you can imagine each regiment is a two-dimensional array giving details about the company, names and the scores, and they are stacked one below the other. 
</p>
</div>

In [ ]:
df_2_ind.mean(level='company')

In [ ]:
df_2_ind.mean(level='regiment')

In [ ]:
df_2_ind.mean(level=['regiment','company'])

### Concatenating two or more `Series` to a `DataFrame` with `axis = 1` parameter in `pd.concat()` function

In [ ]:
s1 = pd.Series([1, 2], index=['A', 'B'], name='s1')
s2 = pd.Series([3, 4,5], index=['A', 'B','C'], name='s2')

In [ ]:
s1

In [ ]:
s2

In [ ]:
pd.concat([s1, s2], axis=1)

<div class="alert alert-block alert-info">
<p>
Note that the ``pd.concat()`` function is smart to label the column names with the series variable names. 
</div> 